In [1]:
import os
import time
import pickle

import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn import model_selection
import sklearn.metrics as metrics

In [18]:
FEATURES = ['net_acc_std', 'net_acc_max', 'EDA_tonic_mean', 'EDA_tonic_min',  'EDA_tonic_max', 'label']
STRESS = {
    0: "amusement", 1: "baseline", 2: "stress"
}
STRESS_MAP = {
    1: 0,
    2: 1
}

In [70]:
def prepare_dataset(dataset_name):
    data = pd.read_csv(f'../../data/03_primary/{dataset_name}/combined_subjects.csv')
    df = data.loc[:, data.columns.intersection(FEATURES)]
    df = df[df.label != 0]
    Y_ = df.label.map(STRESS_MAP)
    X_ = df.drop(columns=['label'])
    return model_selection.train_test_split(X_, Y_, test_size=0.25, random_state=42, stratify=Y_)

In [71]:
def train(m, dataset_name):
    X, x, Y, y = prepare_dataset(dataset_name)
    start_time = time.time()
    result = m.fit(X, Y).predict(x)
    print(f"Finished in: {time.time() - start_time:.2f} seconds")
    print("Accuracy: {:.2f}%".format(metrics.accuracy_score(y, result) * 100))
    print("Precision: {0:.2f}".format(metrics.precision_score(y, result, average='macro')))
    print("Recall: {0:.2f}".format(metrics.recall_score(y, result, average='macro')))
    print("F1 Score: {0:.2f}".format(metrics.f1_score(y, result, average='macro')))
    print("R2 Score: {0:.2f}".format(metrics.r2_score(y, result)))

In [72]:
train(xgb.XGBClassifier(objective="binary:logistic", random_state=42), 'WESAD_preprocessed_int10_add10')

Finished in: 0.16 seconds
Accuracy: 94.86%
Precision: 0.94
Recall: 0.95
F1 Score: 0.94
R2 Score: 0.78


In [73]:
train(xgb.XGBClassifier(objective="binary:logistic", random_state=42), 'WESAD_preprocessed_int10_add15')

Finished in: 0.16 seconds
Accuracy: 95.60%
Precision: 0.95
Recall: 0.95
F1 Score: 0.95
R2 Score: 0.81


In [74]:
train(xgb.XGBClassifier(objective="binary:logistic", eta=0.1, random_state=42), 'WESAD_preprocessed_int15_add15')

Finished in: 0.16 seconds
Accuracy: 96.82%
Precision: 0.97
Recall: 0.97
F1 Score: 0.97
R2 Score: 0.86
